In [1]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.set_random_seed(42)

In [2]:

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12844730105292370977
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7360780698
locality {
  bus_id: 1
}
incarnation: 7288376093822242781
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
]


In [1]:
# import data

def open_pickle(path):
    import pickle
    with open(path, 'rb') as f:
        X = pickle.load(f)
    return X

X_train_sequence = open_pickle('../data/imdb/X_train_sequence.pkl')
X_test_sequence = open_pickle('../data/imdb/X_test_sequence.pkl')
y_train_target = np.asarray(open_pickle('../data/imdb/y_train_target.pkl'))
y_test_target = np.asarray(open_pickle('../data/imdb/y_test_target.pkl'))

with open('cv_vocabulary.txt', 'r', encoding='utf-8') as f:
    vocab = f.read().splitlines()

NameError: name 'np' is not defined

### Global variable

In [3]:
EPOCHS = 100
BATCH_SIZE = 64

# Sigmoid connected layer
#### Keras

In [4]:
## Let's try in Keras for now.

from keras.initializers import Constant, glorot_uniform
from keras.layers import Input, Dense
from keras.models import Model

input_layer = Input(shape=(X_train_sequence.shape[1],))
middle_layer = Dense(100, activation='tanh', kernel_initializer=glorot_uniform(seed=42))(input_layer)
# second_middle_layer = Dense(64, activation='relu', kernel_initializer=glorot_uniform(seed=42))(middle_layer)
output = Dense(y_train_target.shape[1], activation='sigmoid', kernel_initializer=glorot_uniform(seed=42))(middle_layer)
# output = Dense(2, activation='sigmoid', kernel_initializer=glorot_uniform(seed=42))(middle_layer)

model = Model(inputs=input_layer, outputs=output)

model.summary()

model.compile(loss='binary_crossentropy',
             metrics=['acc'],
             optimizer='rmsprop')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1315)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               131600    
_________________________________________________________________
dense_2 (Dense)              (None, 1315)              132815    
Total params: 264,415
Trainable params: 264,415
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [5]:
from sklearn.utils import class_weight

class_weights = []

for i in range(y_train_target.shape[1]):
    class_weight_list = class_weight.compute_class_weight('balanced', np.unique(y_train_target[:,i]), y_train_target[:,i])
    class_weights.append(dict(zip(np.unique(y_train_target[:,i]), class_weight_list)))

In [6]:
# for i in class_weights:
#     try: 
#         print('%.5f\t%.2f' %(i[0], i[1]))
#     except KeyError:
#         print('%.5f' %(i[0]))

In [6]:
model.fit(X_train_sequence, y_train_target, validation_split=(1./3), epochs=10, 
          class_weight=class_weights, verbose=1, batch_size=BATCH_SIZE)

Train on 15168 samples, validate on 7584 samples
Epoch 1/10
15168/15168 [==============================] - 3s 188us/step - loss: 0.0547 - acc: 0.9923 - val_loss: 0.0179 - val_acc: 0.9967
Epoch 2/10
15168/15168 [==============================] - 2s 114us/step - loss: 0.0166 - acc: 0.9967 - val_loss: 0.0157 - val_acc: 0.9967
Epoch 3/10
15168/15168 [==============================] - 2s 115us/step - loss: 0.0148 - acc: 0.9969 - val_loss: 0.0144 - val_acc: 0.9970
Epoch 4/10
15168/15168 [==============================] - 2s 114us/step - loss: 0.0136 - acc: 0.9970 - val_loss: 0.0135 - val_acc: 0.9971
Epoch 5/10
15168/15168 [==============================] - 2s 114us/step - loss: 0.0128 - acc: 0.9971 - val_loss: 0.0130 - val_acc: 0.9971
Epoch 6/10
15168/15168 [==============================] - 2s 114us/step - loss: 0.0121 - acc: 0.9972 - val_loss: 0.0126 - val_acc: 0.9972
Epoch 7/10
15168/15168 [==============================] - 2s 114us/step - loss: 0.0116 - acc: 0.9973 - val_loss: 0.0123 - v

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def get_F1(model_):
    y_pred_train = (model_.predict(X_train_sequence)>0.5).astype(np.float32)
    y_pred_test = (model_.predict(X_test_sequence)>0.5).astype(np.float32)
    
    for i in range(y_pred_train.shape[1]):
        tr_pre, tr_rec, tr_f, tr_sup = precision_recall_fscore_support(y_train_target[:,i], y_pred_train[:,i], average='binary')
        te_pre, te_rec, te_f, te_sup = precision_recall_fscore_support(y_test_target[:,i], y_pred_test[:,i], average='binary')
        print('%.5f\t%.5f\t%.5f\t%.5f\t%.5f\t%.5f' %(tr_pre, tr_rec, tr_f, te_pre, te_rec, te_f))


In [9]:
get_F1(model)

/home/anneke/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.86585	0.66355	0.75132	0.52459	0.38554	0.44444
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.89362	0.76829	0.82623	0.55660	0.39597	0.46275
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.40000	0.04545	0.08163	0.00000	0.00000	0.00000
0.85276	0.75543	0.80115	0.63584	0.58201	0.60773
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.11111	0.02439	0.04000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.82456	0.69118	0.75200	0.62963	0.44444	0.52107
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.87879	0.73418	0.80000	0.62385	0.42236	0.50370
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.47826	0.20886	0.29075	0.07292	0.05385	0.06195
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.17391	0.03941	0.06426	0.01639	0.00889	0.01153
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	

0.78065	0.59901	0.67787	0.38919	0.32579	0.35468
0.75294	0.47059	0.57919	0.15957	0.12712	0.14151
0.14286	0.03846	0.06061	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.03571	0.01587	0.02198
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.49412	0.13419	0.21106	0.12281	0.06583	0.08571
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.62921	0.37333	0.46862	0.13873	0.08541	0.10573
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.80000	0.09091	0.16327	0.07143	0.01852	0.02941
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.69231	0.38764	0.49700	0.14130	0.10656	0.12150
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.12500	0.02817	0.04598	0.10714	0.03797	0.05607
0.22222	0.07059	0.10714	0.01639	0.01053	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.42857	0.06977	0.12000	0.16667	0.02326	0.04082
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.42857	0.11538	0.18182	0.07018	0.03226	0.04420
0.76471	0.39394	0.52000	0.19298	0.11111	0.14103
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.18750	0.03750	0.06250	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.21311	0.04127	0.06915	0.01974	0.00885	0.01222
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.88000	0.34375	0.49438	0.22222	0.07692	0.11429
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.87692	0.56436	0.68675	0.48214	0.30682	0.37500
0.00000	0.00000	0.00000	0.00000	0.00000	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.69325	0.47083	0.56079	0.30808	0.23372	0.26580
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.86765	0.59596	0.70659	0.54217	0.39474	0.45685
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.01449	0.00833	0.01058
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.53125	0.26154	0.35052	0.08511	0.06250	0.07207
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.45455	0.10417	0.16949	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.64407	0.41304	0.50331	0.37931	0.25581	0.30556
0.00000	0.00000	0.00000	0.00000	0.00000	

0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.04348	0.01163	0.01835	0.03279	0.02500	0.02837
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.48485	0.16471	0.24588	0.06625	0.02983	0.04114
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.20000	0.06452	0.09756
0.66667	0.08000	0.14286	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.60465	0.43697	0.50732	0.23445	0.23333	0.23389
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	0.00000
0.00000	0.00000	0.00000	0.00000	0.00000	

In [7]:
score = model.evaluate(X_test_sequence, y_test_target)

22701/22701 [==============================] - 2s 75us/step


In [11]:
score

[0.0224803939677831, 0.9959737134790595]

In [8]:
score

[0.011748266042993382, 0.9972810759023866]

In [13]:
# score

In [9]:
preds = (model.predict(X_train_sequence)>0.5).astype(np.int32)

In [10]:
preds.shape

(22752, 1315)

In [16]:
preds_sum = np.sum(preds,axis=1)

for i in range(np.max(preds_sum)):
    print(i, np.sum(preds_sum==i))

0 1498
1 4880
2 6550
3 5314
4 2741
5 1077
6 255
7 140
8 84
9 52
10 31
11 24
12 21
13 20
14 6
15 2
16 3
17 8
18 5
19 5
20 3
21 5
22 2
23 1
24 2
25 4
26 0
27 3
28 1
29 0
30 1
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 1
44 0
45 0
46 0
47 1
48 0
49 0
50 0
51 1
52 0
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 2
71 0
72 0
73 0
74 0
75 1
76 0
77 0
78 0
79 1
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 1
108 1
109 0
110 0
111 0
112 1
113 0
114 1
115 0
116 0
117 0
118 0
119 0
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 1
140 0
141 0
142 0
143 0
144 0
145 0
146 0
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 1
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
18

In [17]:
np.sum(preds_sum==0)/len(preds_sum)

0.06584036568213783

#### Features which predict non zeros

In [18]:
output_sum = np.sum(preds, axis=0)
cnt = 0
for i in range(len(output_sum)):
    if output_sum[i] == 0:
        continue
    print('%d\t%s\t%d' %(i, vocab[i], output_sum[i]))
    cnt += 1

1	1/10	82
3	10/10	141
6	2	5
7	2/10	163
12	4/10	1
16	7/10	114
18	8/10	132
21	9/10	53
23	a	2537
26	about	104
29	absolutely	24
33	acted	7
34	acting	171
35	action	14
36	actor	157
37	actress	17
39	actually	14
46	after	14
55	all	93
56	almost	4
60	also	37
61	although	4
62	always	13
63	am	8
64	amazing	302
67	among	1
70	an	449
71	and	2143
75	annoying	864
76	another	12
79	any	24
91	are	334
95	as	250
98	aspect	8
99	at	461
108	avoid	131
109	award	11
112	awful	779
117	bad	847
118	badly	512
125	be	381
126	beautiful	165
127	beautifully	107
130	because	42
132	becomes	2
133	been	31
134	before	1
139	being	34
143	best	4537
150	bit	36
156	book	1
158	boring	387
159	both	8
169	but	347
171	by	50
176	camera	1
178	can	32
185	cast	69
198	character	46
202	cheap	211
207	choice	3
211	cinematography	1
226	comedy	19
234	complete	25
235	completely	3
251	could	19
279	day	1
288	definitely	7
294	despite	5
300	dialogue	12
301	did	46
309	direction	10
310	director	9
311	disappointed	208
312	disappointing	125
313	disappoint

In [19]:
cnt

327

In [11]:
output_sum = np.sum(preds, axis=0)
cnt = 0
for i in range(len(output_sum)):
    if output_sum[i] == 0:
        continue
    print('%d\t%s\t%d' %(i, vocab[i], output_sum[i]))
    cnt += 1

1	1/10	53
3	10/10	166
7	2/10	172
16	7/10	125
18	8/10	151
21	9/10	3
23	a	781
34	acting	9
36	actor	7
64	amazing	370
70	an	215
71	and	150
75	annoying	862
91	are	4
95	as	2
99	at	122
108	avoid	130
112	awful	782
117	bad	854
118	badly	549
125	be	70
126	beautiful	124
127	beautifully	77
143	best	4543
158	boring	408
169	but	7
185	cast	1
202	cheap	258
301	did	1
311	disappointed	223
312	disappointing	142
316	do	156
332	dull	10
360	enjoyable	150
361	enjoyed	43
373	even	11
376	ever	48
383	excellent	677
410	fantastic	372
416	favorite	31
432	film	187
454	for	62
468	friend	101
472	fun	181
474	funny	2223
483	gem	58
510	great	1342
515	guy	2
538	her	14
548	his	85
564	how	1
571	i	484
582	in	5
592	insult	65
600	is	773
602	it	37
613	job	11
618	just	1
683	loved	30
716	mediocre	8
742	most	3
747	movie	345
755	my	16
776	noir	38
781	not	241
793	of	1512
815	oscar	7
829	part	2
839	perfectly	93
840	performance	42
867	poor	376
879	predictable	241
918	recommended	1
921	refreshing	13
1026	so	51
1030	solid	279
1031	some

In [12]:
cnt

101

#### Features which predict all zeros

In [20]:


for i in range(len(output_sum)):
    if output_sum[i] != 0:
        continue
    print('%d\t%s' %(i, vocab[i]))

0	1
2	10
4	100
5	15
8	20
9	3
10	30
11	4
13	5
14	6
15	7
17	8
19	80's
20	9
22	90
24	ability
25	able
27	above
28	absolute
30	accent
31	across
32	act
38	actual
40	adaptation
41	add
42	added
43	admit
44	adult
45	adventure
47	again
48	against
49	age
50	ago
51	agree
52	ain
53	air
54	alien
57	alone
58	along
59	already
65	america
66	american
68	amount
69	amusing
72	animal
73	animated
74	animation
77	answer
78	anti
80	anyone
81	anything
82	anyway
83	apart
84	apparently
85	appeal
86	appear
87	appearance
88	appeared
89	appears
90	appreciate
92	around
93	art
94	artist
96	aside
97	ask
100	atmosphere
101	attack
102	attempt
103	attention
104	attractive
105	audience
106	available
107	average
110	away
111	awesome
113	b
114	baby
115	back
116	background
119	band
120	barely
121	based
122	basic
123	basically
124	battle
128	beauty
129	became
131	become
135	begin
136	beginning
137	begins
138	behind
140	believable
141	believe
142	ben
144	better
145	between
146	beyond
147	big
148	biggest
149	bill
151	bizarre
15

In [21]:
preds.shape

(22752, 1315)

### Tensorflow

In [22]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, X_train_sequence.shape[1]], name='review')
Y = tf.placeholder(tf.float32, [None, y_train_target.shape[1]], name='label')

W = {
    'hidden': tf.Variable(tf.random_normal([X_train_sequence.shape[1], 100])),
    'output': tf.Variable(tf.random_normal([100,y_train_target.shape[1]]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([100], mean=1.0)),
    'output': tf.Variable(tf.random_normal([y_train_target.shape[1]], mean=1.0))
}


hidden = tf.matmul(X, W['hidden']) + biases['hidden']  # hidden layer
hidden = tf.nn.tanh(hidden)

output_ = tf.matmul(hidden, W['output']) + biases['output']  # outputs
# split_layer = tf.split(output_,num_or_size_splits=N_OUTPUTS,axis=1)
preds = tf.nn.sigmoid(output_)

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=output_,
                                                             labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

correct_preds = tf.equal(tf.cast(tf.greater(preds, tf.constant(0.5)), tf.float32), 
                         Y)

accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32), axis=0)
saver = tf.train.Saver()

In [23]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(EPOCHS):
        total_loss = 0
        
        _, loss_per_epoch = sess.run([optimizer, loss], 
                           feed_dict={X: X_train_sequence, Y: y_train_target})
        
        print('Epoch', epoch, 'out of', EPOCHS, 'loss:', loss_per_epoch)
    
    accuracy_test = sess.run(accuracy, 
                             feed_dict={X: X_test_sequence, Y: y_test_target})
    
#     prediction = sess.run(correct_preds, 
#                          feed_dict={X: X_test_sequence, Y:y_test_target})
#     print('Accuracy {0}'.format(accuracy_test))
#     print('output matrix', prediction)
    print(accuracy_test)
#     save_path = saver.save(sess, "./fully_connected_model.ckpt")
# writer.close()

Epoch 0 out of 100 loss: 4.302203
Epoch 1 out of 100 loss: 4.2758937
Epoch 2 out of 100 loss: 4.249629
Epoch 3 out of 100 loss: 4.223417
Epoch 4 out of 100 loss: 4.197256
Epoch 5 out of 100 loss: 4.171144
Epoch 6 out of 100 loss: 4.1450715
Epoch 7 out of 100 loss: 4.119032
Epoch 8 out of 100 loss: 4.093014
Epoch 9 out of 100 loss: 4.0670075
Epoch 10 out of 100 loss: 4.041
Epoch 11 out of 100 loss: 4.01498
Epoch 12 out of 100 loss: 3.988934
Epoch 13 out of 100 loss: 3.9628508
Epoch 14 out of 100 loss: 3.936716
Epoch 15 out of 100 loss: 3.9105177
Epoch 16 out of 100 loss: 3.8842437
Epoch 17 out of 100 loss: 3.8578818
Epoch 18 out of 100 loss: 3.8314204
Epoch 19 out of 100 loss: 3.8048475
Epoch 20 out of 100 loss: 3.7781527
Epoch 21 out of 100 loss: 3.751326
Epoch 22 out of 100 loss: 3.724356
Epoch 23 out of 100 loss: 3.697235
Epoch 24 out of 100 loss: 3.6699529
Epoch 25 out of 100 loss: 3.642501
Epoch 26 out of 100 loss: 3.6148722
Epoch 27 out of 100 loss: 3.5870593
Epoch 28 out of 100 l

In [24]:
accuracy_test.shape

(1315,)

In [25]:
(accuracy_test[:50]>0.7).astype(int)

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1])